In [170]:
import math
import numpy as np
import wandb
from tqdm import tqdm
from torchvision import transforms
from utils import get_data, accuracy, save_Yte, augment_dataset, array_to_tensor
from kernels import *
from models import MultiKRR
from feature import hog
from sklearn.model_selection import train_test_split 

In [171]:
Xtr, Xte, Ytr = get_data()

## Hyperparameter tuning

In [172]:
Xtr_, Xval_, Ytr_, Yval_ = train_test_split(Xtr, Ytr, test_size=0.1, random_state=123)
Xtr_tensor_ = array_to_tensor(Xtr_)
Xval_tensor_ = array_to_tensor(Xval_)

In [173]:
hog_fun = lambda img: hog(img,
    cells_per_block=3,
    normalization='L2-Hys',
)
Xtr_hog_ = np.array([hog_fun(img) for img in tqdm(Xtr_tensor_, desc="Computing HOG")])
Xval_hog_ = np.array([hog_fun(img) for img in Xval_tensor_])


Computing HOG: 100%|██████████| 4500/4500 [00:13<00:00, 327.66it/s]


In [174]:
krr = MultiKRR(kernel=GHIKernel(beta=1.).kernel, lambd=1e-4)
krr.fit(Xtr_hog_, Ytr_)

Fitting: 100%|██████████| 10/10 [00:33<00:00,  3.35s/it]


In [175]:
X = Xtr_hog_
Y = Xval_hog_
(n_x, m_x) = np.shape(X)
(n_y, m_y) = np.shape(Y)
if m_x != m_y: raise TypeError('Check input dimensions')
K = np.zeros((n_y,n_x))
for i in range(n_x):
    for j in range(n_y):
        K[j,i] = np.minimum(np.power(np.abs(X[i,:]), 1.), np.power(np.abs(Y[j,:]), 1.)).sum()

In [176]:
pred = np.array([k.predict(K) for k in krr.KRR_list]).T
pred = np.argmax(pred, axis=1)
print('accuracy: ', accuracy(Yval_,pred)) 

accuracy:  0.2


In [178]:
h_wave = WaveletKernel(a=1.).h
krr_wave = MultiKRR(kernel=WaveletKernel(a=1.).kernel, lambd=1e-4)
krr_wave.fit(Xtr_hog_, Ytr_)

Fitting: 100%|██████████| 10/10 [00:32<00:00,  3.21s/it]


In [179]:
X = Xtr_hog_
Y = Xval_hog_
(n_x, m_x) = np.shape(X)
(n_y, m_y) = np.shape(Y)
if m_x != m_y: raise TypeError('Check input dimensions')
K_wave = np.zeros((n_y,n_x))
for i in range(n_x):
    for j in range(n_y):
        K_wave[j,i] = np.prod(h_wave((X[i,:]-Y[j,:])/1.))

In [181]:
pred_wave = np.array([k.predict(K_wave) for k in krr_wave.KRR_list]).T
pred_wave = np.argmax(pred_wave, axis=1)
print('accuracy: ', accuracy(Yval_,pred_wave))  

accuracy:  0.376


In [ ]:
def get_sweep_id(parameters):
    """https://docs.wandb.ai/guides/sweeps/configuration"""
    sweep_config = {
        'method': "bayes",
        'metric': {
          'name': 'accuracy',
          'goal': 'maximize'
        },
        'parameters': parameters
    }
    sweep_id = wandb.sweep(sweep_config, project='kernel-challenge-final')

    return sweep_id

In [6]:
import math

def one_run():
    wandb.init(project='kernel-challenge-final', entity='theodumont', tags=["HOG+KRR"])
    # kernel =============================================================================
    kernel_name = wandb.config.kernel_name
    if kernel_name == "GaussianKernel":
        kernel = GaussianKernel(sigma=wandb.config.sigma).kernel
    elif kernel_name == "LogKernel":
        kernel = LogKernel(d=wandb.config.d).kernel
    elif kernel_name == "GHIKernel":
        kernel = GHIKernel(beta=wandb.config.beta).kernel
    elif kernel_name == "WaveletKernel":
        pass
    
    # model ==============================================================================
    krr = MultiKRR(
        kernel=kernel,
        lambd=wandb.config.lambd,
    )
    krr.fit(Xtr_hog_, Ytr_)
    wandb.run.summary["accuracy"] = accuracy(Yval_, krr.predict(Xval_hog_))

In [8]:
parameters = {
    'lambd': {
        'distribution': 'log_uniform',
        'min': math.log(1e-5),
        'max': math.log(1e-3),
        # 'value': 1e-4
    },
    'cells_per_block': {
        'value': 3,
    },
    'normalization': {
        'value': 'L2-Hys',
    },
    # kernel -----------------------------------------------------------------------------
    'kernel_name': {
        'value': "GHIKernel",
        # 'values': ["GaussianKernel", "WaveletKernel", "LogKernel", "GHIKernel"],
    },
    # 'd': {
    #     'values': [1,2],
    # },
    'beta': {
        'distribution': 'log_uniform',
        'min': math.log(1e-1),
        'max': math.log(1e1),
    },
}
sweep_id = get_sweep_id(parameters)
%env WANDB_SILENT=True

# sweep_id = "2rx2v24s"
wandb.agent(sweep_id, function=one_run)

Create sweep with ID: e5enugkd
Sweep URL: https://wandb.ai/theodumont/kernel-challenge-final/sweeps/e5enugkd
env: WANDB_SILENT=True


wandb: Agent Starting Run: dqyul4is with config:
wandb: 	cells_per_block: 3
wandb: 	d: 1
wandb: 	kernel_name: LogKernel
wandb: 	lambd: 1.2653316374256873e-05
wandb: 	normalization: L2-Hys
Fitting:  50%|█████     | 5/10 [00:50<00:54, 10.82s/it]

## Predictions

In [5]:
Xtr_tensor = array_to_tensor(Xtr)
Xte_tensor = array_to_tensor(Xte)
hog_fun = lambda img: hog(img,
    cells_per_block=3,
    normalization='L2-Hys',
)
Xtr_hog = np.array([hog_fun(img) for img in tqdm(Xtr_tensor, desc="Computing HOG")])
Xte_hog = np.array([hog_fun(img) for img in Xte_tensor])
krr = MultiKRR(kernel=LogKernel(d=2).kernel, lambd=4e-5)
krr.fit(Xtr_hog, Ytr)
hog_fun = lambda img: hog(img,
    cells_per_block=3,
    normalization='L2-Hys',
)
print(f"accuracy on training set: {accuracy(Ytr, krr.predict(Xtr_hog))}")
Yte = krr.predict(Xte_hog)
save_Yte(Yte, model_name="HOG_Log_MultiKRR_1")

Fitting: 100%|██████████| 10/10 [01:02<00:00,  6.29s/it]


accuracy on training set: 1.0
